# Proposal Document Search
Given a keyword, search post-processed proposals (in .json file format).

In [5]:
'''
USER INPUTS - Change these variables to make code run as desired
    keyword: Keyword to search for in past proposals
    directory: File directory location of the proposal files. The files must be .json files.
Both inputs need to be strings (surrounded by single or double quotes).
'''
keyword = 'amphitheatre'
directory = '../jsons/'


# Import libraries
import json
import os
import pandas as pd
import re


def create_df(directory): 
    ''' Import files and create DataFrame.
        Inputs:
            directory - File directory location of the proposal (json) files
        Outputs:
            df - DataFrame containing the data for all the files
    '''
    df = pd.DataFrame(columns=["Filename", "Header", "Text"])

    # Iterate over files in the directory & remove staff members/places 
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)

        # Dump to file
        with open(f, "r") as json_file: 
            data = json.load(json_file)

        filename = filename[0:-5]

        for key in data:
            df = df.append({"Header":key, "Text":data[key], "Processed_Text":data[key].lower(), 
                            "Filename":filename}, ignore_index=True)

    # Remove punctuation from processed text
    df['Processed_Text'] = df['Processed_Text'].map(lambda x: re.sub('[,\.!?&]', '', x))

    return df

  
def keyword_search(keyword, df):
    ''' Keyword search, searches through the dataframe column Processed_Text.
        Inputs:
            keyword - Word (a string) to search the files for
            df - DataFrame containing the data for all the files
        Outputs:
            occurances - List of row numbers (in df) corresponding to files that contain the keyword 
    ''' 
    keyword = keyword.lower()
    keyword = re.sub('[,\.!?&]', '', keyword)
    
    occurances = []
    for index, row in df.iterrows():
        if keyword in row["Processed_Text"]:
            occurances.append(index)            
    return occurances


def print_found_files(occurances, df):
    ''' Prints the file name that contains the keyword.
        Inputs:
            occurances - List of row numbers (in df) corresponding to files that contain the keyword
            df - DataFrame containing the data for all the files
    '''
    print("======= Files with the search term: =======")
    files = set()
    for r in occurances:
        series = df.iloc[r, [0]]
        files.add(series["Filename"])
        
    for f in files:
        print(f)
    print()
    return


def print_found_text(occurances, df):
    ''' Prints the file text containing the keyword.
        Inputs: 
            occurances - List of row numbers (in df) corresponding to files that contain the keyword
            df - DataFrame containing the data for all the files
    '''
    print("======= Text sections with the search term: =======")
    for r in occurances:
        series = df.iloc[r, [0, 1, 2]]
        print(series["Filename"], "\n", series["Header"], "\n", series["Text"], "\n", sep="")
    return


def main(keyword, directory):
    '''Main driver function.'''
    df = create_df(directory)
    occurances = keyword_search(keyword, df)
    print_found_files(occurances, df)
    print_found_text(occurances, df)
    return

main(keyword, directory)

======= Files with the search term: =======
21.10046 Construction Mngt & Inspection

======= Text sections with the search term: =======
21.10046 Construction Mngt & Inspection
Featured Experience for Rebecca Berner, PE
Rebecca brings an extensive amount of successful project experience, from service to a vast diversity of local government clients, as demonstrated in the following projects. Additionally, Rebecca has worked on construction management, program management, construction inspection, technical requirements, and other related documents in numerous cities throughout Colorado and in other states. Rebecca has significant experience on parks and public recreation facilities, including: Red Rocks Park (Denver Parks), CO: Parking lot development and paving, roadway improvements, drainage installation, trails, sidewalks, and ADA and restroom improvements at and around the amphitheatre Denver Public Schools, Southmoor Elementary School, CO: Playground improvements including walks, dr